In [1]:
import os
import numpy as np
import pandas as pd
import csv

In [4]:
datapath99 = ('../data/cycogs/cycogsgenomes.tsv') # basic information on each genome
datapath3 = ('../data/cycogs/ortholog-metadata.csv') # matches geneID to cycog number
REFS = '../data/img_data_cycog6/'

genomes_df = pd.read_csv(datapath99, sep='\t')
cycogs_df = pd.read_csv(datapath3)

In [23]:
cycog = 60000046
target_df = cycogs_df[cycogs_df.OrthologID == cycog]
target_df

,MappingName,OrthologID,GenomeName,GeneID,Annotation
40882,WH8016_2507493065,60000046,WH8016,2507493065,UDP-glucuronate 4-epimerase
40883,AG-442-B03_2717726087,60000046,AG-442-B03,2717726087,UDP-glucuronate 4-epimerase
40884,scB245a_521K15_2653032043,60000046,scB245a_521K15,2653032043,UDP-glucuronate 4-epimerase
40885,P-SSM2_638311613,60000046,P-SSM2,638311613,UDP-glucuronate 4-epimerase
40886,AG-402-M23_2667747375,60000046,AG-402-M23,2667747375,UDP-glucuronate 4-epimerase
...,...,...,...,...,...
41823,AG-363-I21_2667895488,60000046,AG-363-I21,2667895488,UDP-glucuronate 4-epimerase
41824,AG-418-G18_2717711922,60000046,AG-418-G18,2717711922,UDP-glucuronate 4-epimerase
41825,AG-347-M18_2667707121,60000046,AG-347-M18,2667707121,UDP-glucuronate 4-epimerase
41826,AG-418-L19_2717319812,60000046,AG-418-L19,2717319812,UDP-glucuronate 4-epimerase


In [28]:
for index, row in target_df.iterrows():
    if row['GeneID'] == 2667747375:
        print(row['GenomeName'])


AG-402-M23


In [43]:
cycog = 60000046
target_df = cycogs_df[cycogs_df.OrthologID == cycog]
ID_list = target_df['GeneID'].tolist()
cycog_gene_ids = []
all_parts = []

for genome in os.listdir(REFS):
    if not os.path.isdir(os.path.join(REFS, genome)):
        continue
        # Path to the GFF file for the current genome
    gff_path = os.path.join(REFS, genome, f'{genome}.gff')
    with open(gff_path, 'r') as file:
        linenumber = 0
        for line in file:
            parts = line.strip().split()
            if len(parts) > 7:  # Ensure there are enough parts to parse start and end
                try:
                    start = int(parts[3])
                    end = int(parts[4])
                    # annot = parts[8]
                except ValueError:
                    continue  # Skip lines that do not have valid integers in the expected columns
                gene_id = int(parts[8].split('=')[1].split(';')[0])
                cycog_gene_ids.append(gene_id)
                if gene_id in ID_list:
                    for index, row in target_df.iterrows():
                        if row['GeneID'] == gene_id:
                            genome_name = row['GenomeName']
                            parts.append(genome_name)
                    all_parts.append(parts)

parts_df = pd.DataFrame(all_parts)
parts_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Ga0171917_105,img_core_v400,CDS,100577,101677,.,-,0,ID=2717709056;locus_tag=Ga0171917_105141;produ...,"4,6-dehydratase",AG-418-D13,None,None,None
1,Ga0171917_105,img_core_v400,CDS,111709,112776,.,-,0,ID=2717709065;locus_tag=Ga0171917_105150;product=,dTDP-glucose,"4,6-dehydratase",AG-418-D13,None,None
2,Ga0171917_105,img_core_v400,CDS,113090,114121,.,+,0,ID=2717709067;locus_tag=Ga0171917_105152;produ...,4-epimerase,AG-418-D13,None,None,None
3,Ga0171906_103,img_core_v400,CDS,34400,35446,.,-,0,ID=2717690209;locus_tag=Ga0171906_10345;produc...,4-epimerase,AG-412-G19,None,None,None
4,Ga0078671_11,img_core_v400,CDS,1544108,1545145,.,-,0,ID=2646429362;locus_tag=Ga0078671_111751;produ...,4-epimerase,WH8020,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,NC_006883,img_core_v400,CDS,205819,206724,.,+,0,ID=638311613;locus_tag=PSSM2_270;product=nucle...,epimerase,P-SSM2,None,None,None
942,Ga0171865_107,img_core_v400,CDS,33524,34534,.,+,0,ID=2717285216;locus_tag=Ga0171865_10741;product=,dTDP-glucose,"4,6-dehydratase",AG-402-N08,None,None
943,Ga0171865_107,img_core_v400,CDS,81900,82910,.,-,0,ID=2717285257;locus_tag=Ga0171865_10782;produc...,4-epimerase,AG-402-N08,None,None,None
944,Ga0171856_105,img_core_v400,CDS,19462,20448,.,+,0,ID=2717274398;locus_tag=Ga0171856_10524;produc...,4-epimerase,AG-402-C09,None,None,None


In [ ]:
filtered_df = parts_df.iloc[:, [0, 3, 4, 6]]

# Display the new DataFrame
filtered_df[0].tolist()


In [44]:
if gene_id in ID_list:
    for index, row in target_df.iterrows():
        if row['GeneID'] == gene_id:
            genome_name = row['GenomeName']
            print(genome_name)